In [1]:
import pandas as pd
import os
import urllib
import requests
import wget

ModuleNotFoundError: No module named 'wget'

# Download the data

If `wget` is not installed on your machine, you may want to try `curl URL > file.txt` or the `urllib` package in python.

In [2]:
%%bash
mkdir data_ghcn
cd data_ghcn
wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2012.csv.gz
gzip -d 2012.csv.gz

Couldn't find program: 'bash'


In [3]:
url='https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2012.csv.gz'
response = requests.get(url)
with open(os.path.join("data", "2012.csv"), 'wb') as f:
    f.write(response.content)

# Data Preprocessing

In [4]:
def get_vals(line):
    ls = line.split(',')
    station = ls[0]
    time = ls[1]
    val = float(ls[3])
    return [station, time, val]

def get_stations(filename='data/ghcnd-stations.txt'):
    df = pd.read_csv(filename, '/t', header=None)
    df = df[0].str.split(expand=True)[[0, 1, 2, 3]]
    df.columns = ['Station', 'Latitude', 'Longitude', 'Elevation']
    return df

def process_year(year, stations, col='TAVG', basedir='data/'):
    tavg = []
    with open(os.path.join(basedir, "%s.csv" % year)) as h:
        l = h.readline()
        while l:
            if col in l:
                v = get_vals(l)
                if v[0] in stations.Station.values:
                    tavg.append(get_vals(l))
            l = h.readline()
    df_tavg = pd.DataFrame(tavg, columns=['Station', 'Date', col])
    df_merged = df_tavg.merge(stations, left_on='Station', right_on='Station', how='left')
    df_merged['Date'] = df_merged['Date'].apply(pd.Timestamp)
    for c in ['Latitude', 'Longitude', col, 'Elevation']:
        df_merged[c] = df_merged[c].astype(float)
    return df_merged[['Station', 'Date', col, 'Latitude', 'Longitude', 'Elevation']]

In [6]:
stations = get_stations()
stations.head()

<ipython-input-4-7cda4c0d42fe>:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filename, '/t', header=None)


,Station,Latitude,Longitude,Elevation
0,ACW00011604,17.1167,-61.7833,10.1
1,ACW00011647,17.1333,-61.7833,19.2
2,AE000041196,25.3330,55.5170,34.0
3,AEM00041194,25.2550,55.3640,10.4
4,AEM00041217,24.4330,54.6510,26.8


In [19]:
df1 = process_year('2012', stations, col='TAVG')
df1

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 16: character maps to <undefined>

In [20]:
stations = get_stations()
df1 = process_year('2012', stations, col='TAVG')
stations = stations[stations.Station.isin(df1.Station)]

C:\Users\emman\anaconda3\envs\base_analysis\lib\site-packages\ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 16: character maps to <undefined>

In [ ]:
df2 = process_year('2012', stations, col='PRCP')

In [ ]:
df1.head()

In [ ]:
'USS0022E07S','USS0022E10S','USS0022E11S','USS0021E07S'

In [ ]:
df = df1.merge(df2[['Station', 'Date', 'PRCP']], on=['Station', 'Date'])

df.loc[(df['Station']=='USS0022E07S')|(df['Station']=='USS0022E10S')|(df['Station']=='USS0022E11S')|(df['Station']=='USS0021E07S')]
df.loc[(df['Station']=='USS0022E07S')]
df
#df.to_csv('data_ghcn/daily_global_weather_2020.csv')
df